In [35]:
#Imports

import os
import numpy as np
import cv2
from random import randint
import matplotlib.pyplot as plt

from PIL import Image

import torch

import torchvision.transforms as transforms
import numpy as np

import models
import dataprep
import evalmetrics
import torch.nn as nn
import custom_dataset

from torch.utils.data import Dataset, DataLoader, random_split

In [36]:
#Checking if GPU is available

torch.cuda.is_available()

True

In [37]:
#Preparing Train Test and Validation Data

data_dir = 'SWED_sample/train/'
dataset = custom_dataset.CustomDataset(data_dir)

#Using the split 80% train 10% validation and 10% test
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

#train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])



In [ ]:
#Saving the subset splits

# Save the subsets
torch.save(train_dataset, 'trainc_dataset.pth')
torch.save(val_dataset, 'valc_dataset.pth')
torch.save(test_dataset, 'testc_dataset.pth')

# Load the subset
#loaded_subset_dataset = torch.load('subset_dataset.pth')

In [38]:
#Loading dataset splits (if already saved)

train_dataset = torch.load('trainc_dataset.pth')
val_dataset = torch.load('valc_dataset.pth')
test_dataset = torch.load('testc_dataset.pth')

In [5]:
#Generating Dataloaders compatible with keras models

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)
torch.cuda.empty_cache()


In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#model = models.AttU_Net(img_ch=3,output_ch=1)

## LOADING EUNET 
#model = models.U_Net(img_ch=3,output_ch=1)
model = models.EdgeU1_Net(img_ch=3,output_ch=1)
        
# Defining  loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
#criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_epochs = 30

# Set the device to use for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


######################## Training model on train dataset #####################################

# Train the model
model.to(device)

#Tracking Losses
losst_list = []
lossv_list = []

for epoch in range(num_epochs):
    train_loss = 0.0
    
    #Train the model
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        #print(inputs.shape)
        labels = labels.to(device)
        #print('labels are')
        #print(labels.shape)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and update
        loss.backward()
        optimizer.step()

        # Print statistics
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)
    losst_list.append(avg_train_loss)
            
    #Validate the training process
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
        avg_val_loss = val_loss / len(val_loader)
    lossv_list.append(avg_val_loss)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
    
        
        #print(f"Epoch number{i}")
        #print(f"[Accuracy]{evalmetrics.get_accuracy(outputs,labels,threshold=0.5)}")
        #print(f"[Sensitivity]{evalmetrics.get_sensitivity(outputs,labels,threshold=0.5)}")
        #print(f"[Specificity]{evalmetrics.get_specificity(outputs,labels,threshold=0.5)}")
        #print(f"[Precision]{evalmetrics.get_precision(outputs,labels,threshold=0.5)}")
        #print(f"[F1]{evalmetrics.get_F1(outputs,labels,threshold=0.5)}")
        #print(f"[JS]{evalmetrics.get_JS(outputs,labels,threshold=0.5)}") 
        #print(f"[DC]{evalmetrics.get_DC(outputs,labels,threshold=0.5)}")

Epoch [1/30], Train Loss: 0.1794, Val Loss: 0.0564
Epoch [2/30], Train Loss: 0.0771, Val Loss: 0.0494
Epoch [3/30], Train Loss: 0.0698, Val Loss: 0.0939
Epoch [4/30], Train Loss: 0.0545, Val Loss: 0.0401
Epoch [5/30], Train Loss: 0.0470, Val Loss: 0.0325
Epoch [6/30], Train Loss: 0.0506, Val Loss: 0.0261
Epoch [7/30], Train Loss: 0.0338, Val Loss: 0.0253
Epoch [8/30], Train Loss: 0.0411, Val Loss: 0.0230
Epoch [9/30], Train Loss: 0.0369, Val Loss: 0.0261
Epoch [10/30], Train Loss: 0.0358, Val Loss: 0.0237
Epoch [11/30], Train Loss: 0.0361, Val Loss: 0.0231
Epoch [12/30], Train Loss: 0.0353, Val Loss: 0.0244
Epoch [13/30], Train Loss: 0.0354, Val Loss: 0.0283
Epoch [14/30], Train Loss: 0.0338, Val Loss: 0.0494
Epoch [15/30], Train Loss: 0.0303, Val Loss: 0.0215
Epoch [16/30], Train Loss: 0.0278, Val Loss: 0.0208
Epoch [17/30], Train Loss: 0.0300, Val Loss: 0.0338
Epoch [18/30], Train Loss: 0.0294, Val Loss: 0.0210
Epoch [19/30], Train Loss: 0.0374, Val Loss: 0.0232
Epoch [20/30], Train 

# Save and Load model & Progress

In [23]:
import pickle


# Open a file for writing
with open('train_losses_eunet2.pkl', 'wb') as f:
    # Use pickle to dump the list to the file
    pickle.dump(losst_list, f)
    

# Open a file for writing
with open('val_losses_eunet2.pkl', 'wb') as f:
    # Use pickle to dump the list to the file
    pickle.dump(lossv_list, f)

In [24]:
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': epoch,
    # add any other relevant information
}

torch.save(checkpoint, 'eunet_trained2.pth')

In [25]:
torch.cuda.empty_cache()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(losst_list, label='Train Loss')
plt.plot(lossv_list, label='Validation Loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('AttU Net')
plt.legend()
plt.show()

# Evaluation

In [30]:
torch.cuda.empty_cache()
device = torch.device("cpu")

In [ ]:
################# Load AU-Net
checkpoint = torch.load('aunet_trained2.pth',map_location=torch.device('cpu'))

# Create a new instance of your model
model = models.AttU_Net(img_ch=3,output_ch=1)
model.to(device)

# Load the model state from the checkpoint
model.load_state_dict(checkpoint['model_state_dict'])

# Load the optimizer state from the checkpoint, if needed
optimizer = torch.optim.Adam(model.parameters())
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

model.to(device)
model.eval() 

In [33]:

model.to(device)

# Initialize accumulators for each metric
sensitivity_sum = 0
specificity_sum = 0
precision_sum = 0
F1_sum = 0
JS_sum = 0
DC_sum = 0
num_batches = 0

# Loop over the test dataset
for data, labels in test_loader:
    # Perform inference
    
    
    data = data.to(device)
    labels = labels.to(device)
    preds = model(data)

    # Evaluate the predictions
    sensitivity = evalmetrics.get_sensitivity(preds, labels)
    specificity = evalmetrics.get_specificity(preds, labels)
    precision = evalmetrics.get_precision(preds, labels)
    F1 = evalmetrics.get_F1(preds, labels)
    JS = evalmetrics.get_JS(preds, labels)
    DC = evalmetrics.get_DC(preds, labels)

    # Accumulate the metric values
    sensitivity_sum += sensitivity * len(data)
    specificity_sum += specificity * len(data)
    precision_sum += precision * len(data)
    F1_sum += F1 * len(data)
    JS_sum += JS * len(data)
    DC_sum += DC * len(data)
    num_batches += 1




In [34]:
# Compute the average of each metric
sensitivity_avg = sensitivity_sum / len(test_loader.dataset)
specificity_avg = specificity_sum / len(test_loader.dataset)
precision_avg = precision_sum / len(test_loader.dataset)
F1_avg = F1_sum / len(test_loader.dataset)
JS_avg = JS_sum / len(test_loader.dataset)
DC_avg = DC_sum / len(test_loader.dataset)

# Print the average of each metric
print("Average Sensitivity: {:.4f}".format(sensitivity_avg))
print("Average Specificity: {:.4f}".format(specificity_avg))
print("Average Precision: {:.4f}".format(precision_avg))
print("Average F1 Score: {:.4f}".format(F1_avg))
print("Average Jaccard Similarity: {:.4f}".format(JS_avg))
print("Average Dice Coefficient: {:.4f}".format(DC_avg))

Average Sensitivity: 0.9160
Average Specificity: 0.9977
Average Precision: 0.9537
Average F1 Score: 0.9293
Average Jaccard Similarity: 0.8957
Average Dice Coefficient: 0.9293
